### titanic_train.csv 파일을 로드하고, 이를 DataFrame으로 변환

In [1]:
from pyspark.sql import SparkSession

# SparkSession 객체 생성
spark = SparkSession.builder \
    .appName("stock") \
    .getOrCreate()



# pandas 버전
# pandas_df = pd.read_csv('titanic_train.csv', header='infer')

#spark.read.csv() 메소드를 이용하여 csv 파일을 로드하고 DataFrame으로 변환. 
# CSV 파일을 pandas 데이터프레임으로 불러올 때, 'NULL' 문자열을 NaN 값으로 대체하도록 설정
stock_sdf = spark.read.csv('./data/stock_info_0315.csv', header=True, inferSchema=True,  nullValue='NULL')
print('stock sdf type:', type(stock_sdf))

stock_sdf.show(5)

# spark DataFrame을 메모리에 cache
stock_sdf = stock_sdf.cache()

stock sdf type: <class 'pyspark.sql.dataframe.DataFrame'>
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|종목코드|      종목명| 종가|등락률|    시가총액|    기준일|   eps|  per|선행eps|선행per|     bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율| 시가| 고가| 저가|거래량|  거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|  000020|    동화약품| 9350| -0.85|261159244500|2024-03-15| 736.0| 12.7|   null|   null| 13165.0|0.71|     180.0|      1.93|       1681856|        6.02|      27931470|            6.02| 9450| 9450| 9300| 7220

In [2]:
import pandas as pd

na_values = ['NULL']
stock_pdf = pd.read_csv('./data/stock_info_0315.csv', header='infer', na_values=na_values)

stock_pdf.head(5)

,종목코드,종목명,종가,등락률,시가총액,기준일,eps,per,선행eps,선행per,...,시가,고가,저가,거래량,거래대금,기업고유번호,시장구분,종목구분,섹터,업종명
0,000020,동화약품,9350,-0.85,261159244500,2024-03-15,736.0,12.70,NaN,NaN,...,9450,9450,9300,72206,674278570,119195.0,KOSPI,보통주,건강관리,의약품
1,000040,KR모터스,465,0.00,44704386225,2024-03-15,NaN,NaN,NaN,NaN,...,0,0,0,0,0,112378.0,KOSPI,보통주,경기관련소비재,운수장비
2,000050,경방,8470,-1.63,232207336900,2024-03-15,177.0,47.85,NaN,NaN,...,8610,8690,8270,11024,93485040,101628.0,KOSPI,보통주,경기관련소비재,유통업
3,000070,삼양홀딩스,71300,-1.93,610632522300,2024-03-15,9173.0,7.77,NaN,NaN,...,73000,74500,71300,32624,2341408000,126937.0,KOSPI,보통주,소재,기타금융
4,000075,삼양홀딩스우,54900,-0.18,16692784200,2024-03-15,NaN,NaN,NaN,NaN,...,55400,55400,54800,96,5264000,NaN,KOSPI,우선주,NaN,기타금융


In [41]:
# filter의 SQL문 사용시 한글 컬럼을 조건에 넣으면 오류가 발생하기 때문에 컬럼명을 영문으로 변경해야 함 
# 컬럼명 변경
stock_pdf = stock_pdf.rename(columns={
    '종목코드': 'stock_code',
    '종목명': 'stock_name',
    '종가': 'closing_price',
    '등락률': 'price_change',
    '시가총액': 'market_cap',
    '기준일': 'base_date',
    '선행eps': 'leading_eps',
    '선행per': 'leading_per',
    '주당배당금': 'dividend_per_share',
    '배당수익률': 'dividend_yield',
    '외국인보유수량': 'foreign_ownership_quantity',
    '외국인지분율': 'foreign_ownership_ratio',
    '외국인한도수량': 'foreign_limit_quantity',
    '외국인한도소진율': 'foreign_limit_exhaustion_ratio',
    '시가': 'opening_price',
    '고가': 'high_price',
    '저가': 'low_price',
    '거래량': 'trading_volume',
    '거래대금': 'trading_value',
    '기업고유번호': 'company_id',
    '시장구분': 'market_division',
    '종목구분': 'stock_division',
    '섹터': 'sector',
    '업종명': 'industry_name',
})

# spark DataFrame으로 변환 
stock_sdf = spark.createDataFrame(stock_pdf)

### spark DataFrame의 orderBy() 알아보기
* spark DataFrame의 orderBy() 메소드는 1개 이상의 컬럼순으로 정렬할 수 있는 기능. orderBy() 결과는 DataFrame으로 반환. 
* 정렬 컬럼은 문자열, 또는 컬럼 형태로 입력할 수 있으며, 정렬 컬럼이 여러개일 경우 개별 컬럼을 인자로 넣거나 list로도 넣을 수 있음. 
* 오름차순, 내림차순 구분은 ascending=True/False로 구분
* 정렬 컬럼이 여러개 일때 개별 컬럼별로 서로 다른 정렬 옵션을 적용할 경우(예를 들어 컬럼1은 오름차순, 컬럼2는 내림차순) ascending=[True, False]와 같은 형태로 이용.

In [5]:
# stock_name 컬럼으로 오름차순으로 정렬 
stock_pdf_sorted_01 = stock_pdf.sort_values(by=['stock_name'], ascending=True)

# price_change과 stock_name 컬럼으로 내림차순 정렬
stock_pdf_sorted_02 = stock_pdf.sort_values(by=['price_change', 'stock_name'], ascending=False)

# price_change은 내림차순 정렬, stock_name은 오름차순으로 정렬
stock_pdf_sorted_03 = stock_pdf.sort_values(by=['price_change', 'stock_name'], ascending=[False, True])


,stock_code,stock_name,closing_price,price_change,market_cap,base_date,eps,per,leading_eps,leading_per,...,opening_price,high_price,low_price,trading_volume,trading_value,company_id,market_division,stock_division,sector,industry_name
1138,060310,3S,2885,3.41,140028212170,2024-03-15,30.0,96.17,NaN,NaN,...,2770,2885,2755,870995,2479964755,378363.0,KOSDAQ,보통주,산업재,기계·장비
1485,095570,AJ네트웍스,4420,-2.00,200017194780,2024-03-15,201.0,21.99,685.0,6.45,...,4575,4580,4420,195759,874663450,365387.0,KOSPI,보통주,산업재,서비스업
356,006840,AK홀딩스,15600,-1.02,206661951600,2024-03-15,NaN,NaN,NaN,NaN,...,15990,15990,15500,5722,89370090,125080.0,KOSPI,보통주,소재,기타금융
1090,054620,APS,7310,3.25,149081755510,2024-03-15,505.0,14.48,NaN,NaN,...,7100,7410,7000,72073,517280580,296078.0,KOSDAQ,보통주,IT,금융
2110,265520,AP시스템,20800,-2.35,317853556800,2024-03-15,5463.0,3.81,NaN,NaN,...,21250,21250,20750,62176,1299617950,1203808.0,KOSDAQ,보통주,IT,반도체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,000540,흥국화재,4625,-3.65,297122233125,2024-03-15,2142.0,2.16,NaN,NaN,...,4795,4935,4535,468432,2182032780,103176.0,KOSPI,보통주,NaN,보험
38,000545,흥국화재우,6060,1.00,4654080000,2024-03-15,NaN,NaN,NaN,NaN,...,5950,6140,5950,4518,27496580,NaN,KOSPI,우선주,NaN,보험
188,003280,흥아해운,2620,-3.14,629913235380,2024-03-15,94.0,27.87,NaN,NaN,...,2690,2740,2605,1112994,2954941065,167208.0,KOSPI,보통주,산업재,운수창고업
869,037440,희림,7050,1.88,98153448750,2024-03-15,567.0,12.43,736.0,9.58,...,7130,7300,7030,547219,3897157550,236863.0,KOSDAQ,보통주,산업재,기타서비스


,stock_code,stock_name,closing_price,price_change,market_cap,base_date,eps,per,leading_eps,leading_per,...,opening_price,high_price,low_price,trading_volume,trading_value,company_id,market_division,stock_division,sector,industry_name
44,000680,LS네트웍스,5640,29.95,444449010240,2024-03-15,NaN,NaN,NaN,NaN,...,4350,5640,4350,38170724,197276280850,104698.0,KOSPI,보통주,경기관련소비재,유통업
739,030350,드래곤플라이,621,29.92,43090552233,2024-03-15,NaN,NaN,NaN,NaN,...,489,621,484,12930458,7793481076,230036.0,KOSDAQ,보통주,커뮤니케이션서비스,디지털컨텐츠
2072,255220,SG,2765,29.81,146056438325,2024-03-15,80.0,34.56,NaN,NaN,...,2765,2765,2560,12255275,33725481010,963976.0,KOSDAQ,보통주,산업재,비금속
1620,115530,씨엔플러스,345,29.70,23447235000,2024-03-15,26.0,13.27,NaN,NaN,...,268,345,268,14110954,4431400942,660033.0,KOSDAQ,보통주,IT,IT부품
1881,200470,에이팩트,5910,26.01,250359969630,2024-03-15,105.0,56.29,NaN,NaN,...,4705,6050,4700,31252432,178391774655,667425.0,KOSDAQ,보통주,IT,반도체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,313760,윌링스,6750,-14.77,39074946750,2024-03-15,NaN,NaN,NaN,NaN,...,7920,7970,6020,1012533,6649591410,863038.0,KOSDAQ,보통주,에너지,일반전기전자
923,041590,플래스크,1128,-16.44,148565890800,2024-03-15,NaN,NaN,NaN,NaN,...,1300,1329,1070,6142882,7555244904,297448.0,KOSDAQ,보통주,산업재,건설
1819,180640,한진칼,59100,-16.64,3945650688900,2024-03-15,10143.0,5.83,NaN,NaN,...,63900,64300,58700,364168,21890099100,983040.0,KOSPI,보통주,산업재,운수창고업
1009,049470,SGA,403,-20.98,23721486347,2024-03-15,NaN,NaN,NaN,NaN,...,475,475,385,8462102,3469552549,351579.0,KOSDAQ,보통주,IT,소프트웨어


,stock_code,stock_name,closing_price,price_change,market_cap,base_date,eps,per,leading_eps,leading_per,...,opening_price,high_price,low_price,trading_volume,trading_value,company_id,market_division,stock_division,sector,industry_name
44,000680,LS네트웍스,5640,29.95,444449010240,2024-03-15,NaN,NaN,NaN,NaN,...,4350,5640,4350,38170724,197276280850,104698.0,KOSPI,보통주,경기관련소비재,유통업
739,030350,드래곤플라이,621,29.92,43090552233,2024-03-15,NaN,NaN,NaN,NaN,...,489,621,484,12930458,7793481076,230036.0,KOSDAQ,보통주,커뮤니케이션서비스,디지털컨텐츠
2072,255220,SG,2765,29.81,146056438325,2024-03-15,80.0,34.56,NaN,NaN,...,2765,2765,2560,12255275,33725481010,963976.0,KOSDAQ,보통주,산업재,비금속
1620,115530,씨엔플러스,345,29.70,23447235000,2024-03-15,26.0,13.27,NaN,NaN,...,268,345,268,14110954,4431400942,660033.0,KOSDAQ,보통주,IT,IT부품
1881,200470,에이팩트,5910,26.01,250359969630,2024-03-15,105.0,56.29,NaN,NaN,...,4705,6050,4700,31252432,178391774655,667425.0,KOSDAQ,보통주,IT,반도체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,313760,윌링스,6750,-14.77,39074946750,2024-03-15,NaN,NaN,NaN,NaN,...,7920,7970,6020,1012533,6649591410,863038.0,KOSDAQ,보통주,에너지,일반전기전자
923,041590,플래스크,1128,-16.44,148565890800,2024-03-15,NaN,NaN,NaN,NaN,...,1300,1329,1070,6142882,7555244904,297448.0,KOSDAQ,보통주,산업재,건설
1819,180640,한진칼,59100,-16.64,3945650688900,2024-03-15,10143.0,5.83,NaN,NaN,...,63900,64300,58700,364168,21890099100,983040.0,KOSPI,보통주,산업재,운수창고업
1009,049470,SGA,403,-20.98,23721486347,2024-03-15,NaN,NaN,NaN,NaN,...,475,475,385,8462102,3469552549,351579.0,KOSDAQ,보통주,IT,소프트웨어


In [7]:
from pyspark.sql.functions import col 

# orderBy에 컬럼명을 문자열로 지정. 
print("orderBy에 컬럼명을 문자열로 지정하고 내림 차순 정렬")

# select * from stock_sdf order by stock_name desc
stock_sdf.orderBy("stock_name", ascending=False).show(3)

# orderBy에 컬럼명을 컬럼형태로 지정.
print("orderBy에 컬럼명을 DataFrame['컬럼명'] 컬럼형태로 오름 차순 정렬")

# select * from stock_sdf order by stock_name asc
stock_sdf.orderBy(stock_sdf['stock_name'], ascending=True).show(3) 

print('orderBy에 컬럼명을 DataFrame.컬럼명 컬럼형태로 내림 차순 정렬')
stock_sdf.orderBy(stock_sdf.stock_name, ascending=False).show(3)

print("orderBy에 컬럼명을 col('컬럼명') 컬럼형태로 오름 차순 정렬")
stock_sdf.orderBy(col('stock_name'), ascending=True).show(3)


orderBy에 컬럼명을 문자열로 지정하고 내림 차순 정렬
+----------+----------+-------------+------------+------------+----------+-----+-----+-----------+-----------+------+----+------------------+--------------+--------------------------+-----------------------+----------------------+------------------------------+-------------+----------+---------+--------------+-------------+----------+---------------+--------------+------+-------------+
|stock_code|stock_name|closing_price|price_change|  market_cap| base_date|  eps|  per|leading_eps|leading_per|   bps| pbr|dividend_per_share|dividend_yield|foreign_ownership_quantity|foreign_ownership_ratio|foreign_limit_quantity|foreign_limit_exhaustion_ratio|opening_price|high_price|low_price|trading_volume|trading_value|company_id|market_division|stock_division|sector|industry_name|
+----------+----------+-------------+------------+------------+----------+-----+-----+-----------+-----------+------+----+------------------+--------------+--------------------------+------

In [10]:
from pyspark.sql.functions import col

print("orderBy에 여러개의 컬럼명을 문자열로 지정하고 내림 차순 정렬")

# select * from titanic_sdf order by price_change desc, stock_name desc
stock_sdf.orderBy("price_change", "stock_name", ascending=False).show(3)
stock_sdf.orderBy(["price_change", "stock_name"], ascending=False).show(3)

print("orderBy에 여러개의 컬럼명을 컬럼형태로 지정하고 내림 차순 정렬")
stock_sdf.orderBy(col("price_change"), col("stock_name"), ascending=False).show(3)


orderBy에 여러개의 컬럼명을 문자열로 지정하고 내림 차순 정렬
+----------+------------+-------------+------------+------------+----------+----+-----+-----------+-----------+------+----+------------------+--------------+--------------------------+-----------------------+----------------------+------------------------------+-------------+----------+---------+--------------+-------------+----------+---------------+--------------+------------------+-------------+
|stock_code|  stock_name|closing_price|price_change|  market_cap| base_date| eps|  per|leading_eps|leading_per|   bps| pbr|dividend_per_share|dividend_yield|foreign_ownership_quantity|foreign_ownership_ratio|foreign_limit_quantity|foreign_limit_exhaustion_ratio|opening_price|high_price|low_price|trading_volume|trading_value|company_id|market_division|stock_division|            sector|industry_name|
+----------+------------+-------------+------------+------------+----------+----+-----+-----------+-----------+------+----+------------------+--------------+-

In [12]:
# orderBy에 여러개의 컬럼명을 지정하고 서로 다른 방식으로 정렬하기
from pyspark.sql.functions import col 

print("orderBy에 여러개의 컬럼명을 문자열로 지정하고 서로 다른 방식으로 정렬 ")

# select * from stock_sdf order by price_change desc, stock_name asc
stock_sdf.orderBy('price_change', 'stock_name', ascending=[False, True]).show(3)


print("orderBy에 여러개의 컬럼명을 컬럼형태로 지정하고 서로 다른 방식으로 정렬 ")
stock_sdf.orderBy(col('price_change'), col('stock_name'), ascending=[False, True]).show(3)

# 개별 컬럼별로 asc(), desc()를 적용. 
stock_sdf.orderBy(col('price_change').desc(), col('stock_name').asc()).show(3) 

orderBy에 여러개의 컬럼명을 문자열로 지정하고 서로 다른 방식으로 정렬 
+----------+------------+-------------+------------+------------+----------+----+-----+-----------+-----------+------+----+------------------+--------------+--------------------------+-----------------------+----------------------+------------------------------+-------------+----------+---------+--------------+-------------+----------+---------------+--------------+------------------+-------------+
|stock_code|  stock_name|closing_price|price_change|  market_cap| base_date| eps|  per|leading_eps|leading_per|   bps| pbr|dividend_per_share|dividend_yield|foreign_ownership_quantity|foreign_ownership_ratio|foreign_limit_quantity|foreign_limit_exhaustion_ratio|opening_price|high_price|low_price|trading_volume|trading_value|company_id|market_division|stock_division|            sector|industry_name|
+----------+------------+-------------+------------+------------+----------+----+-----+-----------+-----------+------+----+------------------+----------

In [13]:
# orderBy()와 동일한 메소드로 sort()를 제공. 
stock_sdf.sort(col('price_change').desc(), col('stock_name').asc()).show(3)

+----------+------------+-------------+------------+------------+----------+----+-----+-----------+-----------+------+----+------------------+--------------+--------------------------+-----------------------+----------------------+------------------------------+-------------+----------+---------+--------------+-------------+----------+---------------+--------------+------------------+-------------+
|stock_code|  stock_name|closing_price|price_change|  market_cap| base_date| eps|  per|leading_eps|leading_per|   bps| pbr|dividend_per_share|dividend_yield|foreign_ownership_quantity|foreign_ownership_ratio|foreign_limit_quantity|foreign_limit_exhaustion_ratio|opening_price|high_price|low_price|trading_volume|trading_value|company_id|market_division|stock_division|            sector|industry_name|
+----------+------------+-------------+------------+------------+----------+----+-----+-----------+-----------+------+----+------------------+--------------+--------------------------+------------

### spark DataFrame에 aggregation 메소드 적용하기
* pandas DataFrame은 DataFrame 객체에서 aggregation 메소드를 많이 가질 수 있음(DataFrame.count(), DataFrame.max())
* pandas DataFrame은 DataFrame 객체에 aggregation 메소드를 적용 시 DataFrame에 속한 전체 컬럼들에 모두 aggregation 메소드를 적용
* spark DataFrame은 DataFrame 객체에서 aggregation 메소드를 별로 가지고 있지 않음. count() 메소드 정도... 
* spark DataFrame에 aggregation 메소드를 적용 시에는 pyspark.sql.functions 모듈의 max, min, sum 등의 함수를 이용해야함.

In [15]:
print('#### pandas dataframe count() aggregation ####')
print(stock_pdf.count())

print('#### pandas dataframe max() aggregation ####')
print(stock_pdf.max())

print('#### pandas dataframe count() aggregation 결과 type ####')
print(type(stock_pdf.count()))

#### pandas dataframe count() aggregation ####
stock_code                        2671
stock_name                        2671
closing_price                     2671
price_change                      2671
market_cap                        2671
base_date                         2671
eps                               1600
per                               1600
leading_eps                        511
leading_per                        511
bps                               2381
pbr                               2381
dividend_per_share                2622
dividend_yield                    2622
foreign_ownership_quantity        2671
foreign_ownership_ratio           2671
foreign_limit_quantity            2671
foreign_limit_exhaustion_ratio    2671
opening_price                     2671
high_price                        2671
low_price                         2671
trading_volume                    2671
trading_value                     2671
company_id                        2554
market_division  

/tmp/ipykernel_42567/2166524444.py:5: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(stock_pdf.max())


In [17]:
print(stock_pdf[['market_division', 'market_cap']].max())

market_division              KOSPI
market_cap         431615278365000
dtype: object


In [18]:
# spark DataFrame에 count() aggregation을 적용하면 DataFrame의 Record 건수 반환. 
print('count 결과:', stock_sdf.count()) # select count(*) from titanic_sdf

count 결과: 2671


In [19]:
# 하지만 count() 가 아닌 다른 aggregation 함수를 DataFrame에 적용하면 오류 발생.이는 SQL과 유사
# 이는 count() aggregation 함수가 가진 특수성. 다른 aggregation 함수들은 어떤 컬럼을 aggregation 할지 명시해줘야 함. 
# count()외의 다른 aggregation 함수, 예를 들어 max(), min()등은 pyspark.sql.functions 모듈에 별도로 구현되어 있음. 
stock_sdf.max() # select max() from titanic_sdf 와 같은 SQL을 구문 오류. 

AttributeError: 'DataFrame' object has no attribute 'max'

In [21]:
from pyspark.sql.functions import max, sum, min

# spark DataFrame에 count()를 제외하고 max(), min(), sum(), avg()와 같은 aggregate 메소드를 바로 호출할 수 없으며, select()메소드 내에서 호출되어야 함. 
stock_sdf_max = stock_sdf.select(max('market_cap')) # select max(market_cap) from stock_sdf
print(stock_sdf_max.show())
print(type(stock_sdf_max)) # max() aggregation은 단 한개의 값을 반환하지만 DataFrame으로 반환. 

+---------------+
|max(market_cap)|
+---------------+
|431615278365000|
+---------------+

None
<class 'pyspark.sql.dataframe.DataFrame'>


### spark DataFrame의 groupBy() 알아 보기
* pandas DataFrame의 groupby(by='group_by_컬럼명') 수행 시 group_by_컬럼명 레벨로 group by 된 DataFrameGroupBy 객체 반환하고 여기에 aggregation 메소드 적용. 
* spark DataFrame도 groupBy('group_by_컬럼명') 수행 시 group_by_컬럼명 레벨로 group by 된 GroupedData 객체 반환하고 여기에 aggregation 메소드 적용.
* pandas DataFrameGroupBy 객체에 agg() 메소드를 이용하여 서로 다른 컬럼에 서로 다른 aggregation 함수 적용 가능
* spark GroupedData 객체도 agg() 메소드를 이용하여 서로 다른 컬럼에 서로 다른 aggregation 함수 적용 가능
* spark groupBy()는 pandas groupby()의 특징과 SQL의 특징을 함께 가짐.

In [26]:
# pandas DataFrame에 groupby()메소드 호출 시 DataFrameGroupBy 객체 반환. 
stock_pdf_groupby = stock_pdf.groupby(by='market_division')
print('pandas DataFrame의 groupby() 적용 결과 type:', type(stock_pdf_groupby))

# Group by 된 pandas DataFrameGroupBy 객체에 count()를 적용 시 group by 된 컬럼값 레벨로 모든 컬럼들의 count() 수행. 
print('\n#### group by 레벨로 모든 컬럼에 count 적용 #### ')
print(stock_pdf.groupby(by='market_division').count())

print('\n#### group by 레벨로 특정 컬럼에 aggregation 적용 #### ')
# Group by 된 pandas DataFrameGroupBy 객체에 특정 컬럼에 aggregation 을 적용하려면 해당 컬럼을 ['컬럼명'] 추출하여 aggregation 함수 적용. 
# select max(market_cap) from stock_pdf group by market_division
print(stock_pdf.groupby(by='market_division')['market_cap'].max()) 

# pandas DataFrameGroupBy 객체에 여러 컬럼에 동일 aggregation 을 적용하려면 해당 컬럼들을 [['컬럼명1', '컬럼명2']]로 추출하여 aggregation 함수 적용. 
print('\n####  group by 레벨로 여러 컬럼에 동일 aggregation 적용 #### ')
 # select max(market_cap), max(price_change) from stock_pdf group by market_cap
print(stock_pdf.groupby(by='market_division')[['market_cap', 'price_change']].max())

# Group by 된 DataFrameGroupBy 객체에 서로 다른 컬럼에 서로 다른 aggregation 함수를 적용하려면 agg() 메소드를 사용. 
# agg()메소드 내부에 인자는 dictionary 형태로 적용 컬럼명과 적용 aggregation 함수 기재
print('\n####  group by 레벨로 여러개의 aggregation 함수를 서로 다른 컬럼에 적용 #### ')
agg_format = {'market_cap':'mean', 'price_change':'max', 'trading_volume':'mean'}
print(stock_pdf.groupby(by='market_division').agg(agg_format))

pandas DataFrame의 groupby() 적용 결과 type: <class 'pandas.core.groupby.generic.DataFrameGroupBy'>

#### group by 레벨로 모든 컬럼에 count 적용 #### 
                 stock_code  stock_name  closing_price  price_change  \
market_division                                                        
KOSDAQ                 1718        1718           1718          1718   
KOSPI                   953         953            953           953   

                 market_cap  base_date  eps  per  leading_eps  leading_per  \
market_division                                                              
KOSDAQ                 1718       1718  990  990          222          222   
KOSPI                   953        953  610  610          289          289   

                 ...  foreign_limit_exhaustion_ratio  opening_price  \
market_division  ...                                                  
KOSDAQ           ...                            1718           1718   
KOSPI            ...                             

In [27]:
# pandas DataFrame의 value_counts()는 Series에 적용시 해당 series내의 값 별로 건수를 구함. 
print(stock_pdf['stock_division'].value_counts())

보통주    2419
우선주     117
스팩       86
기타       49
Name: stock_division, dtype: int64


In [30]:
# pandas 의 value_counts()의 대응될 수 있는 groupBy() 메소드. Spark DataFrame에 groupBy() 적용 시 GroupedData Object 반환.
# GroupedData Object에 count()외에 min(), max(), avg(), sum() 등 다양한 aggregation 메소드를 호출하여 group by, aggregation 결과 DataFrame 반환. 
stock_sdf.groupBy('stock_division').count().show() # select stock_division, count(*) from stock_sdf group by stock_division

print('spark DataFrame groupBy type:', type(stock_sdf.groupBy('stock_division')))
print('spark GroupedData의 aggregation 메소드 적용 결과 type:', stock_sdf.groupBy('stock_division').count()) 

+--------------+-----+
|stock_division|count|
+--------------+-----+
|        보통주| 2419|
|        우선주|  117|
|          기타|   49|
|          스팩|   86|
+--------------+-----+

spark DataFrame groupBy type: <class 'pyspark.sql.group.GroupedData'>
spark GroupedData의 aggregation 메소드 적용 결과 type: DataFrame[stock_division: string, count: bigint]


In [31]:
# spark DataFrame의 orderBy()메소드를 적용하여 group by 결과 건수 descending 으로 정렬 
stock_sdf.groupBy('stock_division').count().orderBy('count', ascending=False).show()

+--------------+-----+
|stock_division|count|
+--------------+-----+
|        보통주| 2419|
|        우선주|  117|
|          스팩|   86|
|          기타|   49|
+--------------+-----+



In [32]:
#GroupedData 에 count()가 아니고 다른 aggregation 메소드를 적용 시 pandas DataFrame의 groupby와 유사하게 group by된 컬럼 레벨로 전체 컬럼에 대해서 aggregation을 적용. 
stock_sdf.groupBy('stock_division').max().show() 

+--------------+------------------+-----------------+---------------+--------+--------+----------------+----------------+--------+--------+-----------------------+-------------------+-------------------------------+----------------------------+---------------------------+-----------------------------------+------------------+---------------+--------------+-------------------+------------------+---------------+
|stock_division|max(closing_price)|max(price_change)|max(market_cap)|max(eps)|max(per)|max(leading_eps)|max(leading_per)|max(bps)|max(pbr)|max(dividend_per_share)|max(dividend_yield)|max(foreign_ownership_quantity)|max(foreign_ownership_ratio)|max(foreign_limit_quantity)|max(foreign_limit_exhaustion_ratio)|max(opening_price)|max(high_price)|max(low_price)|max(trading_volume)|max(trading_value)|max(company_id)|
+--------------+------------------+-----------------+---------------+--------+--------+----------------+----------------+--------+--------+-----------------------+---------

In [33]:
# group by 레벨로 특정 컬럼에 aggregation 적용. max('컬럼명')과 같이 aggregation 메소드 내부에 인자로 컬러명 입력
# select max(market_cap) from stock_sdf group by stock_division
stock_sdf.groupBy('stock_division').max('market_cap').show() 

#GroupedData에서 aggregation 메소드 호출 시 오직 문자열 컬럼명만 가능. 컬럼형 인자 입력은 오류 발생. 
stock_sdf.groupBy('stock_division').max(col('Age')).show()

+--------------+---------------+
|stock_division|max(market_cap)|
+--------------+---------------+
|        보통주|431615278365000|
|        우선주| 51018975400000|
|          기타|  5599790411750|
|          스팩|    73651200000|
+--------------+---------------+



TypeError: Column is not iterable

In [34]:
# 여러 컬럼으로 Group by 규정할 때 개별 컬럼명을 입력하거나, list 형태로 입력 가능. 
# select max(market_cap) from stock_sdf group by market_division, stock_division
stock_sdf.groupBy('market_division', 'stock_division').max('market_cap').show()
stock_sdf.groupBy(['market_division', 'stock_division']).max('market_cap').show()

+---------------+--------------+---------------+
|market_division|stock_division|max(market_cap)|
+---------------+--------------+---------------+
|          KOSPI|        보통주|431615278365000|
|          KOSPI|        우선주| 51018975400000|
|         KOSDAQ|        보통주| 25086044736000|
|         KOSDAQ|        우선주|    11861393880|
|          KOSPI|          기타|  5599790411750|
|         KOSDAQ|          스팩|    73651200000|
|         KOSDAQ|          기타|   920199963200|
+---------------+--------------+---------------+

+---------------+--------------+---------------+
|market_division|stock_division|max(market_cap)|
+---------------+--------------+---------------+
|          KOSPI|        보통주|431615278365000|
|          KOSPI|        우선주| 51018975400000|
|         KOSDAQ|        보통주| 25086044736000|
|         KOSDAQ|        우선주|    11861393880|
|          KOSPI|          기타|  5599790411750|
|         KOSDAQ|          스팩|    73651200000|
|         KOSDAQ|          기타|   920199963200|
+-----

In [35]:
### 여러개의 aggregation 함수를 적용할 경우는 agg()메소드 내에서 개별 aggregation 함수를 명시 해야함. 

from pyspark.sql.functions import max, avg, sum, min

# select max(market_cap), min(market_cap), sum(market_cap), avg(market_cap) from stock_sdf group by stock_division
stock_sdf.groupBy('stock_division').agg(max('market_cap'), min('market_cap'), sum('market_cap'), avg('market_cap')).show() 

+--------------+---------------+---------------+----------------+--------------------+
|stock_division|max(market_cap)|min(market_cap)| sum(market_cap)|     avg(market_cap)|
+--------------+---------------+---------------+----------------+--------------------+
|        보통주|431615278365000|     7721265564|2496516216812539|1.032044736177155...|
|        우선주| 51018975400000|     3530573280|  72525119381545|6.198728152268804E11|
|          기타|  5599790411750|    13969290160|  17833516449083|3.639493152874081...|
|          스팩|    73651200000|     7021500000|   1312502575000|1.526165784883721E10|
+--------------+---------------+---------------+----------------+--------------------+



In [37]:
#아래와 같이 개별 aggregation 함수 결과 컬럼에 별도의 컬럼명을 alias('새로운 컬럼명')을 활용하여 부여 할 수 있음. 
# agg() 메소드 내에서 aggregation 함수 적용 시에는 col('컬럼명')과 같은 컬럼형으로 컬럼명을 지정해도 됨. 
# select max(market_cap) as max_market_cap, min(market_cap) as min_market_cap, sum(market_cap) as sum_market_cap, avg(market_cap) as avg_market_cap from stock_sdf group by stock_division
stock_sdf.groupBy('stock_division').agg(
    max(col('market_cap')).alias('max_market_cap'), min('market_cap').alias('min_market_cap'), \
    sum('market_cap').alias('sum_market_cap'), avg('market_cap').alias('avg_market_cap') \
    ).show()

+--------------+---------------+--------------+----------------+--------------------+
|stock_division| max_market_cap|min_market_cap|  sum_market_cap|      avg_market_cap|
+--------------+---------------+--------------+----------------+--------------------+
|        보통주|431615278365000|    7721265564|2496516216812539|1.032044736177155...|
|        우선주| 51018975400000|    3530573280|  72525119381545|6.198728152268804E11|
|          기타|  5599790411750|   13969290160|  17833516449083|3.639493152874081...|
|          스팩|    73651200000|    7021500000|   1312502575000|1.526165784883721E10|
+--------------+---------------+--------------+----------------+--------------------+



In [39]:
# 아래와 같이 filter()를 적용하여 group by의 aggregation 결과 값을 기준으로 filtering 적용할 수 있음. 
'''
select max(market_cap), min(market_cap), sum(market_cap), avg(market_cap) from stock_sdf 
group by stock_division having max(age) > 80000000000

또는 

select max_age, min_age, sum_avg, avg_age 
from (
      # select max(market_cap) as max_market_cap, min(market_cap) as min_market_cap, sum(market_cap) as sum_market_cap, avg(market_cap) as avg_market_cap from stock_sdf 
      group by stock_division
) where  max_market_cap > 80000000000
'''

stock_sdf.groupBy('stock_division').agg(
    max(col('market_cap')).alias('max_market_cap'), min('market_cap').alias('min_market_cap'), \
    sum('market_cap').alias('sum_market_cap'), avg('market_cap').alias('avg_market_cap') \
    ).filter(col('max_market_cap') > 80000000000).show()

+--------------+---------------+--------------+----------------+--------------------+
|stock_division| max_market_cap|min_market_cap|  sum_market_cap|      avg_market_cap|
+--------------+---------------+--------------+----------------+--------------------+
|        보통주|431615278365000|    7721265564|2496516216812539|1.032044736177155...|
|        우선주| 51018975400000|    3530573280|  72525119381545|6.198728152268804E11|
|          기타|  5599790411750|   13969290160|  17833516449083|3.639493152874081...|
+--------------+---------------+--------------+----------------+--------------------+

